# **BOUNDING BOX REGRESSION WITH RESNET 50**
**WE ARE GOING TO USE CALTECH-101 DATASET, I HAVE UPDATED THE DATASET FOR OUR PURPOSE, ZUERST IMPLEMENTIEREN DIE DATASET**

In [ ]:
#### ZUERST IMPLEMENTIREN WIR DIE BIBLIOTHEKEN, DI WIR BRAUCHEN KÖNNEN:
import os
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import sys
import datetime
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
import pickle
import zipfile # importing the 'zipfile' module

from google.colab import files
from google.colab import drive
import matplotlib.patches as patches

!pip install send2trash

from send2trash import send2trash
import pandas as pd

from keras.callbacks import EarlyStopping
import tensorflow as tf

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import cv2

import cv2
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from keras.models import Sequential
## Import from keras_preprocessing not from keras.preprocessing
## from keras_preprocessing.image import ImageDataGenerator

from tensorflow.keras import optimizers

from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import time
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import tensorflow_datasets as tfds

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

######################################################
############## RESNET50 ##############################
######################################################


import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU

In [ ]:
# https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
# https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_dataframe/
# https://stackoverflow.com/questions/66424141/imagedatagenerator-flow-from-dataframe-multi-output-regression-and-classificatio
# https://stackoverflow.com/questions/50781562/stratified-splitting-of-pandas-dataframe-into-training-validation-and-test-set

##################################################
########### DATA LOADER LINKS ####################
##################################################


In [ ]:
## WE ARE GOING TO IMPORT THE ZIP FILE, WHICH CONSIST THE IMAGES:

drive.mount('/content/drive')

In [ ]:
zip_file_name = "/content/drive/MyDrive/caltech-101_m2.zip"  # Zip dosyasının adını güncelleyin

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall("caltech_files/")

In [ ]:
!pip install send2trash

from send2trash import send2trash

file_path = 'caltech-101_new/101_ObjectCategories/BACKGROUND_Google'

try:
    send2trash(file_path)
    print(f"File '{file_path}' has been moved to the trash.")
except OSError:
    print(f"Failed to delete file '{file_path}'.")

In [ ]:
## LET'S IMPORT THE DATAFRAMES FOR VALIDATION, TEST AND TRAIN

train_path = '/content/drive/My Drive/train_df.csv'
train_df = pd.read_csv(train_path)

val_path = '/content/drive/My Drive/val_df.csv'
val_df = pd.read_csv(val_path)

test_path = '/content/drive/My Drive/test_df.csv'
test_df = pd.read_csv(test_path)

In [ ]:
print(type(train_df))
print(type(test_df))
print(type(val_df))


print(len(train_df))
print(len(test_df))
print(len(val_df))


In [ ]:
train_df.to_csv('/content/train_df.csv', index=False)
val_df.to_csv('/content/val_df.csv', index=False)
test_df.to_csv('/content/test_df.csv', index=False)


In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
val_df.head()

In [ ]:

def plot_eda(dataframe):
    # Count the number of unique classes
    num_classes = len(dataframe['Class Name'].unique())

    # Count the number of images
    num_images = dataframe.shape[0]

    # Count the number of images per class
    images_per_class = dataframe['Class Name'].value_counts()

    # Plotting
    plt.figure(figsize=(12, 12))
    plt.barh(range(num_classes), images_per_class.values)
    plt.yticks(range(num_classes), images_per_class.index)
    plt.xlabel('Number of Images')
    plt.ylabel('Class')
    plt.title('Number of Images per Class')

    # Display the number of images
    for i, v in enumerate(images_per_class.values):
        plt.text(v, i, str(v), color='black', va='center')

    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()


In [ ]:
# Call the function for each dataframe
plot_eda(train_df)
plot_eda(val_df)
plot_eda(test_df)

In [ ]:
# Combine the dataframes
combined_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

def plot_combined_eda(dataframe):
    # Count the number of unique classes
    num_classes = len(dataframe['Class Name'].unique())

    # Count the number of images
    num_images = dataframe.shape[0]

    # Count the number of images per class
    images_per_class = dataframe['Class Name'].value_counts()

    # Plotting
    plt.figure(figsize=(12, 12))
    plt.barh(range(num_classes), images_per_class.values, height=0.5)

    # Adjust the spacing between the y-tick labels
    plt.gca().set_yticks(range(num_classes))
    plt.gca().set_yticklabels(images_per_class.index, ha='right')

    plt.xlabel('Number of Images')
    plt.ylabel('Class')
    plt.title('Number of Images per Class (Combined)')

    # Display the number of images
    for i, v in enumerate(images_per_class.values):
        plt.text(v + 5, i, str(v), color='black', va='center')

    # Display the total number of images
    plt.text(0, num_classes+1, f'Total Images: {num_images}', fontsize=12, fontweight='bold')

    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()

# Call the function for the combined dataframe
plot_combined_eda(combined_df)

### **WE HAVE THE DATAFRAMES FOR TRAINING, VALIDATION AND TEST LET'S CONTINUE WITH TRAINING THE MODEL**

In [ ]:
train_df = pd.read_csv('/content/drive/My Drive/train_df.csv')
val_df = pd.read_csv('/content/drive/My Drive/val_df.csv')
test_df = pd.read_csv('/content/drive/My Drive/test_df.csv')

In [ ]:
column_names = ['Image Path', 'Class Name', 'Image Name', 'Bounding Box', 'Height', 'Width', 'label',
                'normalized', 'Xmin', 'Ymin', 'Xmax', 'Ymax']


train_df.columns = column_names
val_df.columns = column_names
test_df.columns = column_names

In [ ]:
def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    image = image.astype(np.float32) / 255.0
    return image

In [ ]:

def plot_images_with_bboxes(df):
    fig, axes = plt.subplots(1, 5, figsize=(20, 4))
    for i, row in df.head(5).iterrows():
        image_path = row['Image Path']
        class_name = row['Class Name']
        image_name = row['Image Name']
        xmin, ymin, xmax, ymax = row['Xmin']*224, row['Ymin']*224, row['Xmax']*224, row['Ymax']*224

        image = load_image(image_path)

        ax = axes[i]
        ax.imshow(image)
        ax.axis('off')

        # Add bounding box
        bbox = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(bbox)

        ax.set_title(f'Class: {class_name}\nImage: {image_name}')

    plt.show()

# Plot images with bounding boxes for val_df, test_df, train_df
plot_images_with_bboxes(val_df)
plot_images_with_bboxes(test_df)
plot_images_with_bboxes(train_df)

In [ ]:
####################################################
############## MOST IMPORTANT PART #################
####################################################
def resize_image(image, target_size):
    return cv2.resize(image, target_size)

def plot_images(images, boxes, labels, dataset_name):
    fig, axes = plt.subplots(1, len(images), figsize=(12, 4))
    for i, ax in enumerate(axes):
        ax.imshow(images[i])
        xmin, ymin, xmax, ymax = boxes[i]
        ax.add_patch(plt.Rectangle((xmin*224, ymin*224), xmax*224 - xmin*224, ymax*224 - ymin*224, fill=False, color='red', linewidth=2))
        ax.set_title(labels[i])
        ax.axis('off')
    plt.suptitle(f'{dataset_name} Images with Bounding Boxes')
    plt.show()

# Append data from val_df into arrays
val_images = []
val_boxes = []
val_labels = []

for index, row in val_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    val_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    val_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    val_labels.append(label)

# Convert the lists to NumPy arrays
val_images = np.array(val_images, dtype='float32')
val_boxes = np.array(val_boxes, dtype='float32')
val_labels = np.array(val_labels)

# Normalize the images
val_images = val_images / 255.0

# Plot the images, bounding boxes, and class names for the first 4 images from the val_df dataset
plot_images(val_images[:5], val_boxes[:5], val_labels[:5], 'Validation')

# Append data from train_df into arrays
train_images = []
train_boxes = []
train_labels = []

for index, row in train_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    train_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    train_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    train_labels.append(label)

# Convert the lists to NumPy arrays
train_images = np.array(train_images, dtype='float32')
train_boxes = np.array(train_boxes, dtype='float32')
train_labels = np.array(train_labels)

# Normalize the images
train_images = train_images / 255.0

# Plot the images, bounding boxes, and class names for the first 4 images from the train_df dataset
plot_images(train_images[:5], train_boxes[:5], train_labels[:5], 'Train')

# Append data from test_df into arrays
test_images = []
test_boxes = []
test_labels = []

for index, row in test_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    test_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    test_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    test_labels.append(label)

# Convert the lists to NumPy arrays
test_images = np.array(test_images, dtype='float32')
test_boxes = np.array(test_boxes, dtype='float32')
test_labels = np.array(test_labels)

# Normalize the images
test_images = test_images / 255.0

# Plot the images, bounding boxes, and class names for the first 4 images from the test_df dataset
plot_images(test_images[:5], test_boxes[:5], test_labels[:5], 'Test')


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Convert the encoded labels to one-hot encoding
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=101)
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=101)


# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)

flatten = Flatten()(base_model.output)

bboxHead = Dense(1024, activation="relu")(flatten)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)
bboxHead = Dense(1024, activation="relu")(bboxHead)


bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(512, activation="relu")(bboxHead)

bboxHead = Dense(256, activation="relu")(bboxHead)
bboxHead = Dense(256, activation="relu")(bboxHead)
bboxHead = Dense(256, activation="relu")(bboxHead)
bboxHead = Dense(256, activation="relu")(bboxHead)


bboxHead = Dense(128, activation="relu")(bboxHead)
bboxHead = Dense(128, activation="relu")(bboxHead)
bboxHead = Dense(128, activation="relu")(bboxHead)
bboxHead = Dense(128, activation="relu")(bboxHead)


bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(64, activation="relu")(bboxHead)

bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)



softmaxHead = Dense(1024, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(1024, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(1024, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(256, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(256, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(128, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(128, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
class_output = Dense(101, activation='softmax', name='class_output')(softmaxHead)

model = Model(inputs=input_layer, outputs=[class_output, bbox_output], random_state=42)

# Loss function
losses = {
    'class_output': 'categorical_crossentropy',
    'bounding_box': 'mean_squared_error'
}

# Compile the model
model.compile(optimizer='adam', loss=losses)


model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

# Train the model with early stopping
history = model.fit(train_images, {'class_output': train_labels_onehot, 'bounding_box': train_boxes},
                    validation_data=(val_images, {'class_output': val_labels_onehot, 'bounding_box': val_boxes}),
                    epochs=50, batch_size=32)

# Save the trained model
model.save('/content/drive/MyDrive/trained_model.h5')

In [ ]:
import pickle

# Save the label encoder
with open('/content/label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

In [ ]:
# Plot the losses
plt.figure(figsize=(12, 6))
plt.style.use('seaborn')

# Classification Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=14)
plt.legend()

# Bounding Box Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=14)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Perform label encoding on the target labels
test_labels_encoded = label_encoder.transform(test_labels)

# Convert the encoded labels to one-hot encoding
test_labels_onehot = to_categorical(test_labels_encoded, num_classes=101)

# Evaluate the model on the test data
test_loss, test_class_output_loss, test_bbox_output_loss = model.evaluate(test_images, {'class_output': test_labels_onehot, 'bounding_box': test_boxes})

print('Test Loss:', test_loss)
print('Test Classification Loss:', test_class_output_loss)
print('Test Bounding Box Loss:', test_bbox_output_loss)

In [ ]:
import matplotlib.patches as patches
from sklearn.metrics import jaccard_score

# Define a function to calculate the IoU score
def calculate_iou(box1, box2):
    # Extract coordinates from the bounding boxes
    xmin1, ymin1, xmax1, ymax1 = box1
    xmin2, ymin2, xmax2, ymax2 = box2

    # Calculate the coordinates of the intersection rectangle
    xmin_inter = max(xmin1, xmin2)
    ymin_inter = max(ymin1, ymin2)
    xmax_inter = min(xmax1, xmax2)
    ymax_inter = min(ymax1, ymax2)

    # Calculate the area of intersection rectangle
    inter_area = max(0, xmax_inter - xmin_inter + 1) * max(0, ymax_inter - ymin_inter + 1)

    # Calculate the areas of the bounding boxes
    box1_area = (xmax1 - xmin1 + 1) * (ymax1 - ymin1 + 1)
    box2_area = (xmax2 - xmin2 + 1) * (ymax2 - ymin2 + 1)

    # Calculate the IoU score
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

# Define a function to plot test images with predicted bounding box, original bounding box, predicted class, and original class
def plot_test_images(test_images, test_boxes, test_labels, pred_boxes, pred_labels, step=10):
    num_plots = min(len(test_images), 10)
    plt.figure(figsize=(15, 15))
    for i in range(0, len(test_images), step):
        if i // step >= num_plots:
            break
        plt.subplot(5, 2, (i // step) + 1)

        # Denormalize the bounding box coordinates
        x1, y1, x2, y2 = test_boxes[i] * 224
        xmin = int(x1)
        ymin = int(y1)
        xmax = int(x2)
        ymax = int(y2)

        # Denormalize the image array
        image = test_images[i] * 255
        image = image.astype(np.uint8)

        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Original Class: {test_labels[i]}\nPredicted Class: {pred_labels[i]}")

        # Plot original bounding box
        rect = patches.Rectangle((xmin, ymin), (xmax - xmin), (ymax - ymin), linewidth=2, edgecolor='r', facecolor='none')
        plt.gca().add_patch(rect)

        # Denormalize the predicted bounding box coordinates
        x1, y1, x2, y2 = pred_boxes[i] * 224
        xmin = int(x1)
        ymin = int(y1)
        xmax = int(x2)
        ymax = int(y2)

        # Plot predicted bounding box
        rect = patches.Rectangle((xmin, ymin), (xmax - xmin), (ymax - ymin), linewidth=2, edgecolor='g', facecolor='none')
        plt.gca().add_patch(rect)

        # Calculate IoU score
        iou_score = calculate_iou(test_boxes[i] * 224, pred_boxes[i] * 224)
        plt.text(xmin, ymin, f"IoU: {iou_score:.2f}", color='b', fontsize=8, backgroundcolor='w')

    plt.tight_layout()
    plt.show()

# Perform predictions on the test data
preds = model.predict(test_images)
pred_labels = label_encoder.inverse_transform(np.argmax(preds[0], axis=1))
pred_boxes = preds[1]

# Plot every 10th test image with predicted bounding box, original bounding box, predicted class, original class, and IoU score
plot_test_images(test_images, test_boxes, test_labels, pred_boxes, pred_labels, step=1)


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
import numpy as np

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, pred_labels, average=None)

# Get the unique class labels
classes = np.unique(test_labels)

# Set the width of the bars
bar_width = 0.35

# Set the x locations of the bars
index = np.arange(len(classes))

# Set the figure size
plt.figure(figsize=(12, 12))

# Plot the recall scores
plt.bar(index, recall, bar_width, label='Recall', color='skyblue')
plt.xlabel('Class')
plt.ylabel('Score')
plt.title('Recall Scores by Class')
plt.xticks(index, classes, rotation='vertical')

# Add scores as text inside each bar
for i, v in enumerate(recall):
    plt.text(i, v, f"{v:.2f}", color='black', ha='center', va='bottom', fontsize=8)

plt.legend()
plt.tight_layout()
plt.show()

# Set the figure size
plt.figure(figsize=(10, 6))

# Plot the F1 scores
plt.bar(index, f1, bar_width, label='F1 Score', color='lightgreen')
plt.xlabel('Class')
plt.ylabel('Score')
plt.title('F1 Scores by Class')
plt.xticks(index, classes, rotation='vertical')

# Add scores as text inside each bar
for i, v in enumerate(f1):
    plt.text(i, v, f"{v:.2f}", color='black', ha='center', va='bottom', fontsize=8)

plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
class_iou_scores = []

# Calculate the mean IoU score for each class
for class_label in classes:
    # Get the indices of samples belonging to the current class
    class_indices = np.where(test_labels == class_label)[0]

    # Extract the predicted and ground truth bounding boxes for the current class
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    # Calculate the IoU score for each sample
    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    # Calculate the mean IoU score for the current class
    mean_iou_score = np.mean(iou_scores)

    # Append the class label and mean IoU score to the list of class IoU scores
    class_iou_scores.append((class_label, mean_iou_score))

# Sort the mean IoU scores from best to worst
class_iou_scores.sort(key=lambda x: x[1], reverse=True)

# Extract the class labels and mean IoU scores into separate lists
sorted_classes = [x[0] for x in class_iou_scores]
sorted_scores = [x[1] for x in class_iou_scores]

# Calculate the total mean IoU score
total_mean_iou = np.mean(sorted_scores)

# Plot the mean IoU scores for each class
fig, ax = plt.subplots(figsize=(12, 12))
bars = ax.barh(range(len(sorted_classes)), sorted_scores, color='purple')
ax.set_xlabel('Mean IoU Score')
ax.set_ylabel('Class')
ax.set_title('Mean IoU Scores by Class')
ax.set_yticks(range(len(sorted_classes)))
ax.set_yticklabels(sorted_classes)
ax.invert_yaxis()

# Add scores as text inside each bar
for bar, score in zip(bars, sorted_scores):
    ax.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', fontsize=7)

# Add total mean IoU score as text at the top
ax.text(0, -1, f"Total Mean IoU: {total_mean_iou:.2f}", ha='left', va='center', fontsize=12)

# Add a vertical line at x=0.5 for reference
ax.axvline(x=0.5, color='red', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Get the number of images for each class from test_df
class_counts = test_df["Class Name"].value_counts()

# Set the figure size
plt.figure(figsize=(10, 6))

# Plot the number of images per class
class_counts.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Number of Images per Class')

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
import pandas as pd

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Get the unique class labels
classes = np.unique(test_df["Class Name"])

# Sort precision, recall, and F1 scores in descending order
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[classes_sorted]

# Set the figure size
plt.figure(figsize=(12, 12))

# Plot the precision scores
plt.bar(range(len(classes)), precision_sorted, width=0.3, label='Precision', color='lightblue')
plt.xlabel('Class')
plt.ylabel('Score')
plt.title('Precision Scores by Class')
plt.xticks(range(len(classes)), classes_sorted, rotation='vertical')

# Add scores and class counts as text above each bar
for i, v in enumerate(precision_sorted):
    plt.text(i, v, f"{v:.2f}\n({class_counts_sorted[i]})", color='black', ha='center', va='bottom', fontsize=8)

plt.legend()
plt.tight_layout()
plt.show()

# Set the figure size
plt.figure(figsize=(12, 12))

# Plot the recall scores
plt.bar(range(len(classes)), recall_sorted, width=0.3, label='Recall', color='lightgreen')
plt.xlabel('Class')
plt.ylabel('Score')
plt.title('Recall Scores by Class')
plt.xticks(range(len(classes)), classes_sorted, rotation='vertical')

# Add scores and class counts as text above each bar
for i, v in enumerate(recall_sorted):
    plt.text(i, v, f"{v:.2f}\n({class_counts_sorted[i]})", color='black', ha='center', va='bottom', fontsize=8)

plt.legend()
plt.tight_layout()
plt.show()

# Set the figure size
plt.figure(figsize=(12, 12))

# Plot the F1 scores
plt.bar(range(len(classes)), f1_sorted, width=0.3, label='F1 Score', color='orange')
plt.xlabel('Class')
plt.ylabel('Score')
plt.title('F1 Scores by Class')
plt.xticks(range(len(classes)), classes_sorted, rotation='vertical')

# Add scores and class counts as text above each bar
for i, v in enumerate(f1_sorted):
    plt.text(i, v, f"{v:.2f}\n({class_counts_sorted[i]})", color='black', ha='center', va='bottom', fontsize=8)

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
test_df.head()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate the mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Set the figure size and create subplots
fig, ax = plt.subplots(figsize=(12, 12))

# Set the width of the bars
bar_width = 0.8

# Plot the F1 scores
bars = ax.barh(np.arange(len(classes_sorted)), f1_sorted, height=bar_width, color='purple')

# Add F1 scores as text inside each bar
for i, (bar, score) in enumerate(zip(bars, f1_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7)

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(classes_sorted)))
ax.set_yticklabels(classes_sorted, fontsize=7)

# Set x-axis label and limits
ax.set_xlabel('F1 Score', fontsize=10)
ax.set_xlim(0, 1)

# Add class counts as text near the class names
for i, count in enumerate(class_counts_sorted):
    ax.text(-0.07, i, f"  ({count})", va='center', ha='right', fontsize=8)

# Set plot title
ax.set_title('F1 Scores by Class', fontsize=12)

# Remove spines and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(left=False, bottom=False)

# Add a grid to the plot
ax.grid(axis='x', color='lightgray', linestyle='--')

# Adjust layout
fig.tight_layout()

# Invert the y-axis
plt.gca().invert_yaxis()

# Show the plot
plt.show()


In [ ]:
class_iou_scores = []

# Calculate the mean IoU score for each class
for class_label in classes:
    # Get the indices of samples belonging to the current class
    class_indices = np.where(test_labels == class_label)[0]

    # Extract the predicted and ground truth bounding boxes for the current class
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    # Calculate the IoU score for each sample
    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    # Calculate the mean IoU score for the current class
    mean_iou_score = np.mean(iou_scores)

    # Append the class label, mean IoU score, and number of images to the list of class IoU scores
    class_iou_scores.append((class_label, mean_iou_score, len(class_indices)))

# Sort the mean IoU scores from best to worst
class_iou_scores.sort(key=lambda x: x[1], reverse=True)

# Extract the class labels, mean IoU scores, and number of images into separate lists
sorted_classes = [x[0] for x in class_iou_scores]
sorted_scores = [x[1] for x in class_iou_scores]
num_images = [x[2] for x in class_iou_scores]

# Calculate the total mean IoU score
total_mean_iou = np.mean(sorted_scores)

# Plot the mean IoU scores for each class
fig, ax = plt.subplots(figsize=(12, 12))
bars = ax.barh(range(len(sorted_classes)), sorted_scores, color='purple')
ax.set_xlabel('Mean IoU Score')
ax.set_ylabel('Class')
ax.set_title('Mean IoU Scores by Class')
ax.set_yticks(range(len(sorted_classes)))
ax.set_yticklabels(sorted_classes)

for i, count in enumerate(class_counts_sorted):
    ax.text(-0.1, i, f"  ({count})", va='center', ha='right', fontsize=8)

# Add scores as text inside each bar
for bar, score in zip(bars, sorted_scores):
    ax.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', fontsize=7)

# Add total mean IoU score as text at the top
ax.text(0, len(sorted_classes), f"Total Mean IoU: {total_mean_iou:.2f}", ha='left', va='center', fontsize=12)

# Add a vertical line at x=0.5 for reference
ax.axvline(x=0.5, color='red', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate the mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Set the figure size and create subplots
fig, ax = plt.subplots(figsize=(12, 12))

# Set the width of the bars
bar_width = 0.4

# Plot the F1 scores
f1_bars = ax.barh(np.arange(len(classes_sorted)), f1_sorted, height=bar_width, color='purple', label='F1 Score')

# Plot the mean IoU scores
iou_bars = ax.barh(np.arange(len(classes_sorted)) + bar_width, class_iou_scores_sorted, height=bar_width, color='blue', label='Mean IoU')

# Add F1 scores as text inside each bar
for i, (bar, score) in enumerate(zip(f1_bars, f1_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7)

# Add mean IoU scores as text inside each bar
for i, (bar, score) in enumerate(zip(iou_bars, class_iou_scores_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, color='white')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(classes_sorted)) + bar_width / 2)
ax.set_yticklabels(classes_sorted, fontsize=7)

# Set x-axis label and limits
ax.set_xlabel('Scores', fontsize=10)
ax.set_xlim(0, 1)

# Add class counts as text near the class names
for i, count in enumerate(class_counts_sorted):
    ax.text(-0.07, i + bar_width / 2, f"  ({count})", va='center', ha='right', fontsize=8)

# Set plot title
ax.set_title('F1 Scores and Mean IoU Scores by Class', fontsize=12)

# Remove spines and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(left=False, bottom=False)

# Add a grid to the plot
ax.grid(axis='x', color='lightgray', linestyle='--')

# Adjust layout
fig.tight_layout()

# Invert the y-axis
plt.gca().invert_yaxis()

# Show the legend
ax.legend(loc='lower right')

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate the mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Set the figure size and create subplots
fig, ax = plt.subplots(figsize=(12, 12))

# Set the width of the bars
bar_width = 0.4

# Plot the F1 scores
f1_bars = ax.barh(np.arange(len(classes_sorted)), f1_sorted, height=bar_width, color='green', label='F1 Score')

# Plot the mean IoU scores
iou_bars = ax.barh(np.arange(len(classes_sorted)) + bar_width, class_iou_scores_sorted, height=bar_width, color='blue', label='Mean IoU')

# Add F1 scores as text inside each bar
for i, (bar, score) in enumerate(zip(f1_bars, f1_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Add mean IoU scores as text inside each bar
for i, (bar, score) in enumerate(zip(iou_bars, class_iou_scores_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(classes_sorted)) + bar_width / 2)
ax.set_yticklabels(classes_sorted, fontsize=7, fontweight='bold', color='black')

# Set x-axis label and limits
ax.set_xlabel('Scores', fontsize=7, color='red')
ax.set_xlim(0, 1)

# Add class counts as text near the class names
for i, count in enumerate(class_counts_sorted):
    ax.text(-0.1, i + bar_width / 2, f"  ({count})", va='center', ha='right', fontsize=7, fontweight='bold', color='red')

# Set plot title
ax.set_title('F1 Scores and Mean IoU Scores by Class', fontsize=14, fontweight='bold', color='red')

# Remove spines and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(left=False, bottom=False)

# Add a grid to the plot
ax.grid(axis='x', color='lightgray', linestyle='--')

# Show the legend
ax.legend(loc='upper right', fontsize=7)

# Invert the y-axis
plt.gca().invert_yaxis()

# Show the plot
plt.show()


In [ ]:
## WE ARE GOING TO SEPERATE CARSIDE DATASET FROM TRAINING ARRAYS:
## (WE ARE GOING TO ACCOMPLISH OUR EXPERIMENTS WITH 100 DATASET CLASSES)


## * RESNET50 --> MSE --> TRAINABLE = FALSE (TOP ONLY)
## * RESNET50 --> MSE --> TRAINABLE = TRUE (ONLY FULL)
## * RESNET50 --> MSE --> TRAINABLE = FALSE --> THEN --> TRAINABLE = TRUE (TRAIN SAME MODEL WITH SMALLER LEARNING RATE)


## SAME FOR IOU

## THEN SHOE THE RESULTS OF CARSIDE


## CREATE TABLE TO SHOW OUTPUTS IN BETTER WAY

In [ ]:
## LET'S IMPORT THE DATAFRAMES FOR VALIDATION, TEST AND TRAIN

train_path = '/content/drive/My Drive/train_df.csv'
train_df = pd.read_csv(train_path)

val_path = '/content/drive/My Drive/val_df.csv'
val_df = pd.read_csv(val_path)

test_path = '/content/drive/My Drive/test_df.csv'
test_df = pd.read_csv(test_path)

In [ ]:
print(len(train_df))
print(len(test_df))
print(len(val_df))


In [ ]:
import pandas as pd

# Filter rows with 'carside' class and create a new DataFrame
carside_df = train_df[train_df['Class Name'] == 'carside'].copy()

# Remove rows with 'carside' class from the original DataFrame
train_df = train_df[train_df['Class Name'] != 'carside']

# Reset the index of the new DataFrame
carside_df.reset_index(drop=True, inplace=True)

# Reset the index of the original DataFrame
train_df.reset_index(drop=True, inplace=True)

In [ ]:
print(len(carside_df))
print(len(train_df))

In [ ]:
import pandas as pd

# Filter rows with 'carside' class and create a new DataFrame
val_carside_df = val_df[val_df['Class Name'] == 'carside'].copy()

# Remove rows with 'carside' class from the original DataFrame
val_df = val_df[val_df['Class Name'] != 'carside']

# Reset the index of the new DataFrame
val_carside_df.reset_index(drop=True, inplace=True)

# Reset the index of the original DataFrame
val_df.reset_index(drop=True, inplace=True)

In [ ]:
print(len(val_carside_df))
print(len(val_df))

In [ ]:
import pandas as pd

# Filter rows with 'carside' class and create a new DataFrame
test_carside_df = test_df[test_df['Class Name'] == 'carside'].copy()

# Remove rows with 'carside' class from the original DataFrame
test_df = test_df[test_df['Class Name'] != 'carside']

# Reset the index of the new DataFrame
test_carside_df.reset_index(drop=True, inplace=True)

# Reset the index of the original DataFrame
test_df.reset_index(drop=True, inplace=True)

In [ ]:
print(len(test_carside_df))
print(len(test_df))

In [ ]:
import os

# Set the parent directory path
parent_dir = "/content/drive/MyDrive/"

# Set the directory name
directory_name = "project_mark2"

# Combine the parent directory path and directory name
directory_path = os.path.join(parent_dir, directory_name)

# Create the directory if it doesn't exist
os.makedirs(directory_path, exist_ok=True)

# Print the path of the created directory
print("Directory created:", directory_path)

In [ ]:
import os

# Set the parent directory path
parent_dir = "/content/drive/MyDrive/project_mark2"

# Save train_df to CSV
train_df.to_csv(os.path.join(parent_dir, "train_df.csv"), index=False)

# Save val_df to CSV
val_df.to_csv(os.path.join(parent_dir, "val_df.csv"), index=False)

# Save test_df to CSV
test_df.to_csv(os.path.join(parent_dir, "test_df.csv"), index=False)

# Save test_carside_df to CSV
test_carside_df.to_csv(os.path.join(parent_dir, "test_carside_df.csv"), index=False)

# Save val_carside_df to CSV
val_carside_df.to_csv(os.path.join(parent_dir, "val_carside_df.csv"), index=False)

# Save carside_df to CSV
carside_df.to_csv(os.path.join(parent_dir, "carside_df.csv"), index=False)

print("Files saved in directory:", parent_dir)

In [ ]:
#########################################
####### controls for 'carside' ##########

test_carside_df.head()

In [ ]:
val_carside_df.head()

In [ ]:
carside_df.head()

In [ ]:
# Count the occurrences of each class name in the train_df DataFrame
class_counts = train_df['Class Name'].value_counts()

# Check if the class name 'carside' exists in the DataFrame
if 'carside' in class_counts:
    # Get the number of classes with the class name 'carside'
    num_classes_carside = class_counts['carside']
    print(f"The number of classes with the class name 'carside': {num_classes_carside}")
else:
    print("No classes with the class name 'carside' found.")

In [ ]:
# Count the occurrences of each class name in the train_df DataFrame
class_counts = val_df['Class Name'].value_counts()

# Check if the class name 'carside' exists in the DataFrame
if 'carside' in class_counts:
    # Get the number of classes with the class name 'carside'
    num_classes_carside = class_counts['carside']
    print(f"The number of classes with the class name 'carside': {num_classes_carside}")
else:
    print("No classes with the class name 'carside' found.")

In [ ]:
# Count the occurrences of each class name in the train_df DataFrame
class_counts = test_df['Class Name'].value_counts()

# Check if the class name 'carside' exists in the DataFrame
if 'carside' in class_counts:
    # Get the number of classes with the class name 'carside'
    num_classes_carside = class_counts['carside']
    print(f"The number of classes with the class name 'carside': {num_classes_carside}")
else:
    print("No classes with the class name 'carside' found.")

In [ ]:
# Count the occurrences of each class name in the train_df DataFrame
class_counts = carside_df['Class Name'].value_counts()

# Check if the class name 'carside' exists in the DataFrame
if 'carside' in class_counts:
    # Get the number of classes with the class name 'carside'
    num_classes_carside = class_counts['carside']
    print(f"The number of classes with the class name 'carside': {num_classes_carside}")
else:
    print("No classes with the class name 'carside' found.")

In [ ]:
# Count the occurrences of each class name in the train_df DataFrame
class_counts = val_carside_df['Class Name'].value_counts()

# Check if the class name 'carside' exists in the DataFrame
if 'carside' in class_counts:
    # Get the number of classes with the class name 'carside'
    num_classes_carside = class_counts['carside']
    print(f"The number of classes with the class name 'carside': {num_classes_carside}")
else:
    print("No classes with the class name 'carside' found.")

In [ ]:
# Count the occurrences of each class name in the train_df DataFrame
class_counts = test_carside_df['Class Name'].value_counts()

# Check if the class name 'carside' exists in the DataFrame
if 'carside' in class_counts:
    # Get the number of classes with the class name 'carside'
    num_classes_carside = class_counts['carside']
    print(f"The number of classes with the class name 'carside': {num_classes_carside}")
else:
    print("No classes with the class name 'carside' found.")

In [ ]:
############################################################
######## okey we have now train - test - val df's ##########
############################################################
################# we also have carside dataset ###########
###########################################################

############################################################################
########## UPDATE FILE NAMES AND LEARNING RATE FOR FIRST TWO ###############
############################################################################


## **100 classes resnet50 - mse - trainable = Ture**

In [ ]:
# import tensorflow as tf

# def iou_loss(y_true, y_pred):
#     # Extract the normalized coordinates of the predicted and ground truth boxes
#     pred_x1, pred_y1, pred_x2, pred_y2 = tf.unstack(y_pred, axis=1)
#     true_x1, true_y1, true_x2, true_y2 = tf.unstack(y_true, axis=1)

#     # Convert normalized coordinates to absolute coordinates
#     pred_x1 = pred_x1 * 512
#     pred_y1 = pred_y1 * 512
#     pred_x2 = pred_x2 * 512
#     pred_y2 = pred_y2 * 512
#     true_x1 = true_x1 * 512
#     true_y1 = true_y1 * 512
#     true_x2 = true_x2 * 512
#     true_y2 = true_y2 * 512

#     # Calculate the coordinates of the intersection rectangle
#     x1 = tf.maximum(pred_x1, true_x1)
#     y1 = tf.maximum(pred_y1, true_y1)
#     x2 = tf.minimum(pred_x2, true_x2)
#     y2 = tf.minimum(pred_y2, true_y2)

#     # Calculate the area of intersection
#     intersection = tf.maximum(0.0, x2 - x1) * tf.maximum(0.0, y2 - y1)

#     # Calculate the area of predicted and ground truth boxes
#     pred_area = (pred_x2 - pred_x1) * (pred_y2 - pred_y1)
#     true_area = (true_x2 - true_x1) * (true_y2 - true_y1)

#     # Calculate the union area
#     union = pred_area + true_area - intersection

#     # Calculate the IoU
#     iou = intersection / (union + tf.keras.backend.epsilon())

#     # Calculate the IoU loss
#     iou_loss = 1.0 - iou

#     return iou_loss

In [ ]:
# from keras.utils import get_custom_objects
# get_custom_objects().update({'iou_loss': iou_loss})


losses = {
    'class_output': 'categorical_crossentropy',
    'bounding_box': 'mean_squared_error'
}

In [ ]:
####################################################
############## MOST IMPORTANT PART #################
####################################################
def resize_image(image, target_size):
    return cv2.resize(image, target_size)

def plot_images(images, boxes, labels, dataset_name):
    fig, axes = plt.subplots(1, len(images), figsize=(12, 4))
    for i, ax in enumerate(axes):
        ax.imshow(images[i])
        xmin, ymin, xmax, ymax = boxes[i]
        ax.add_patch(plt.Rectangle((xmin*224, ymin*224), xmax*224 - xmin*224, ymax*224 - ymin*224, fill=False, color='red', linewidth=2))
        ax.set_title(labels[i])
        ax.axis('off')
    plt.suptitle(f'{dataset_name} Images with Bounding Boxes')
    plt.show()

# Append data from val_df into arrays
val_images = []
val_boxes = []
val_labels = []

for index, row in val_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    val_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    val_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    val_labels.append(label)

# Convert the lists to NumPy arrays
val_images = np.array(val_images, dtype='float32')
val_boxes = np.array(val_boxes, dtype='float32')
val_labels = np.array(val_labels)

# Normalize the images
val_images = val_images / 255.0



# Append data from train_df into arrays
train_images = []
train_boxes = []
train_labels = []

for index, row in train_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    train_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    train_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    train_labels.append(label)

# Convert the lists to NumPy arrays
train_images = np.array(train_images, dtype='float32')
train_boxes = np.array(train_boxes, dtype='float32')
train_labels = np.array(train_labels)

# Normalize the images
train_images = train_images / 255.0



# Append data from test_df into arrays
test_images = []
test_boxes = []
test_labels = []

for index, row in test_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    test_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    test_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    test_labels.append(label)

# Convert the lists to NumPy arrays
test_images = np.array(test_images, dtype='float32')
test_boxes = np.array(test_boxes, dtype='float32')
test_labels = np.array(test_labels)

# Normalize the images
test_images = test_images / 255.0


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Convert the encoded labels to one-hot encoding
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=100) ## ==> update number of classes
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=100) ## ==> update number of classes

# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)


flatten = Flatten()(base_model.output)


bboxHead = Dense(1024, activation="relu")(flatten)

bboxHead = Dense(512, activation="relu")(bboxHead)

bboxHead = Dense(256, activation="relu")(bboxHead)

bboxHead = Dense(128, activation="relu")(bboxHead)

bboxHead = Dense(64, activation="relu")(bboxHead)

bboxHead = Dense(32, activation="relu")(bboxHead)

bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)




softmaxHead = Dense(1024, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(256, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(128, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

class_output = Dense(100, activation='softmax', name='class_output')(softmaxHead) ## ==> update number of classes



model_resnet_mse_1 = Model(inputs=input_layer, outputs=[class_output, bbox_output])

learning_rate = 1e-4

# # Compile the model for training only the top layers
# model_top_layers.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=losses)


# Compile the model
model_resnet_mse_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=losses)
model_resnet_mse_1.summary()

In [ ]:
from keras.callbacks import EarlyStopping

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)


# Train the model with early stopping
model_resnet_mse_1_history = model_resnet_mse_1.fit(train_images, {'class_output': train_labels_onehot, 'bounding_box': train_boxes},
                    validation_data=(val_images, {'class_output': val_labels_onehot, 'bounding_box': val_boxes}),
                    epochs=300, batch_size=32, callbacks=[callback])

## --> "/content/drive/MyDrive/project_mark2"

# Save the trained model
model_resnet_mse_1.save('/content/drive/MyDrive/project_mark2/model_resnet_mse_1_trained_model.h5')

In [ ]:
import pickle

# Save the label encoder
with open('/content/drive/MyDrive/project_mark2/model_resnet_mse_1_label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

In [ ]:
# Plot the losses
plt.figure(figsize=(24, 12))
plt.style.use('seaborn')

# Classification Loss
plt.subplot(1, 2, 1)
plt.plot(model_resnet_mse_1_history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_resnet_mse_1_history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=12)
plt.legend()

# Bounding Box Loss
plt.subplot(1, 2, 2)
plt.plot(model_resnet_mse_1_history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(model_resnet_mse_1_history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=12)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Save the history object to a file using pickle
with open('model_resnet_mse_1_history.pkl', 'wb') as file:
    pickle.dump(model_resnet_mse_1_history.history, file)

In [ ]:
# Save the entire model to a HDF5 file
model_resnet_mse_1.save('/content/drive/MyDrive/project_mark2/model_resnet_mse_1.h5')

In [ ]:
# Save the history object to a file using pickle
with open('/content/drive/MyDrive/project_mark2/model_resnet_mse_1_history.pkl', 'wb') as file:
    pickle.dump(model_resnet_mse_1_history.history, file)


## **100 classes resnet50 - mse - trainable = false**

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Convert the encoded labels to one-hot encoding
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=100) ## ==> update number of classes
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=100) ## ==> update number of classes

# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)


base_model.trainable = False


flatten = Flatten()(base_model.output)


bboxHead = Dense(1024, activation="relu")(flatten)

bboxHead = Dense(512, activation="relu")(bboxHead)

bboxHead = Dense(256, activation="relu")(bboxHead)

bboxHead = Dense(128, activation="relu")(bboxHead)

bboxHead = Dense(64, activation="relu")(bboxHead)

bboxHead = Dense(32, activation="relu")(bboxHead)

bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)




softmaxHead = Dense(1024, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(256, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(128, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

class_output = Dense(100, activation='softmax', name='class_output')(softmaxHead) ## ==> update number of classes



model_resnet_mse_2 = Model(inputs=input_layer, outputs=[class_output, bbox_output])

# Loss function --> defined in first experiment

learning_rate = 1e-4

# Compile the model
model_resnet_mse_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=losses)

model_resnet_mse_2.summary()

In [ ]:
## at least we understand how we are going to train them:

In [ ]:
from keras.callbacks import EarlyStopping

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)


# Train the model with early stopping
model_resnet_mse_2_history = model_resnet_mse_2.fit(train_images, {'class_output': train_labels_onehot, 'bounding_box': train_boxes},
                    validation_data=(val_images, {'class_output': val_labels_onehot, 'bounding_box': val_boxes}),
                    epochs=300, batch_size=32, callbacks=[callback])

# Save the trained model

## --> "/content/drive/MyDrive/project_mark2"

model_resnet_mse_2.save('/content/drive/MyDrive/project_mark2/model_resnet_mse_2_trained_model.h5')

In [ ]:
import pickle

# Save the label encoder
with open('/content/drive/MyDrive/project_mark2/model_resnet_mse_2_label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

In [ ]:
# Plot the losses
plt.figure(figsize=(24, 12))
plt.style.use('seaborn')

# Classification Loss
plt.subplot(1, 2, 1)
plt.plot(model_resnet_mse_2_history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_resnet_mse_2_history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=12)
plt.legend()

# Bounding Box Loss
plt.subplot(1, 2, 2)
plt.plot(model_resnet_mse_2_history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(model_resnet_mse_2_history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=12)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Save the history object to a file using pickle
with open('model_resnet_mse_2_history.pkl', 'wb') as file:
    pickle.dump(model_resnet_mse_2_history.history, file)

In [ ]:
# Save the entire model to a HDF5 file
model_resnet_mse_2.save('/content/drive/MyDrive/project_mark2/model_resnet_mse_2.h5')

In [ ]:
# Save the history object to a file using pickle
with open('/content/drive/MyDrive/project_mark2/model_resnet_mse_2_history.pkl', 'wb') as file:
    pickle.dump(model_resnet_mse_2_history.history, file)

## **100 classes resnet50 - mse - trainable = false - then - trainable = true with lower learning rate**

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout
from keras.applications import ResNet50

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)            ### --> Canan Hocaya sor ???

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Convert the encoded labels to one-hot encoding
num_classes = 100  # Update with the actual number of classes
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=num_classes)
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=num_classes)

# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)

flatten = Flatten()(base_model.output)

# Top layers for bounding box regression
bboxHead = Dense(1024, activation="relu")(flatten)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(256, activation="relu")(bboxHead)
bboxHead = Dense(128, activation="relu")(bboxHead)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)

# Top layers for classification
softmaxHead = Dense(1024, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(256, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(128, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
class_output = Dense(num_classes, activation='softmax', name='class_output')(softmaxHead)

# Create the first model (train only the top layers)
model_top_layers = Model(inputs=input_layer, outputs=[class_output, bbox_output])

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

learning_rate = 1e-4

# Compile the model for training only the top layers
model_top_layers.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=losses)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)


# Train only the top layers
model_top_layers_history = model_top_layers.fit(train_images, {'class_output': train_labels_onehot, 'bounding_box': train_boxes},
                                                validation_data=(val_images, {'class_output': val_labels_onehot, 'bounding_box': val_boxes}),
                                                epochs=300, batch_size=32, callbacks=[callback])



# Unfreeze the base model's layers
for layer in base_model.layers:
    layer.trainable = True



# Recompile the model with a smaller learning rate for fine-tuning
learning_rate = 1e-5  # Update with the desired learning rate
model_top_layers.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=losses)              #### --> Canan Hocaya sor ???

# Train the model with fine-tuning
model_fine_tuned_history = model_top_layers.fit(train_images, {'class_output': train_labels_onehot, 'bounding_box': train_boxes},
                                                validation_data=(val_images, {'class_output': val_labels_onehot, 'bounding_box': val_boxes}),
                                                epochs=300, batch_size=32, callbacks=[callback])


In [ ]:
# Recompile the model with a smaller learning rate for fine-tuning
learning_rate = 1e-5  # Update with the desired learning rate
model_top_layers.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=losses)              #### --> Canan Hocaya sor ???

# Train the model with fine-tuning
model_fine_tuned_history_2 = model_top_layers.fit(train_images, {'class_output': train_labels_onehot, 'bounding_box': train_boxes},
                                                validation_data=(val_images, {'class_output': val_labels_onehot, 'bounding_box': val_boxes}),
                                                epochs=500, batch_size=128, callbacks=[callback])


In [ ]:
# Save the label encoder
with open('/content/drive/MyDrive/project_mark2/model_top_layers_label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

# Save the entire model to a HDF5 file
model_top_layers.save('/content/drive/MyDrive/project_mark2/model_top_layers.h5')

# Save the history object to a file using pickle
with open('/content/drive/MyDrive/project_mark2/model_fine_tuned_history_2.pkl', 'wb') as file:
    pickle.dump(model_fine_tuned_history_2.history, file)

In [ ]:
# Save the label encoder
with open('/content/drive/MyDrive/project_mark2/model_top_layers_label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

# Save the entire model to a HDF5 file
model_top_layers.save('/content/drive/MyDrive/project_mark2/model_top_layers.h5')

# Save the history object to a file using pickle
with open('/content/drive/MyDrive/project_mark2/model_fine_tuned_history.pkl', 'wb') as file:
    pickle.dump(model_fine_tuned_history_2.history, file)

In [ ]:
# Plot the losses
plt.figure(figsize=(24, 12))
plt.style.use('seaborn')

# Classification Loss
plt.subplot(1, 2, 1)
plt.plot(model_fine_tuned_history_2.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_fine_tuned_history_2.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=12)
plt.legend()

# Bounding Box Loss
plt.subplot(1, 2, 2)
plt.plot(model_fine_tuned_history_2.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(model_fine_tuned_history_2.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=12)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plot the losses
plt.figure(figsize=(24, 12))
plt.style.use('seaborn')

# Classification Loss
plt.subplot(1, 2, 1)
plt.plot(model_fine_tuned_history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_fine_tuned_history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=12)
plt.legend()

# Bounding Box Loss
plt.subplot(1, 2, 2)
plt.plot(model_fine_tuned_history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(model_fine_tuned_history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=12)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
###############################################
################ MSE - 1 ######################
###############################################

In [ ]:
# Perform predictions on the test data
preds = model_resnet_mse_1.predict(test_images)
pred_labels = label_encoder.inverse_transform(np.argmax(preds[0], axis=1))
pred_boxes = preds[1]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)


In [ ]:
import matplotlib.patches as patches
from sklearn.metrics import jaccard_score

# Define a function to calculate the IoU score
def calculate_iou(box1, box2):
    # Extract coordinates from the bounding boxes
    xmin1, ymin1, xmax1, ymax1 = box1
    xmin2, ymin2, xmax2, ymax2 = box2

    # Calculate the coordinates of the intersection rectangle
    xmin_inter = max(xmin1, xmin2)
    ymin_inter = max(ymin1, ymin2)
    xmax_inter = min(xmax1, xmax2)
    ymax_inter = min(ymax1, ymax2)

    # Calculate the area of intersection rectangle
    inter_area = max(0, xmax_inter - xmin_inter + 1) * max(0, ymax_inter - ymin_inter + 1)

    # Calculate the areas of the bounding boxes
    box1_area = (xmax1 - xmin1 + 1) * (ymax1 - ymin1 + 1)
    box2_area = (xmax2 - xmin2 + 1) * (ymax2 - ymin2 + 1)

    # Calculate the IoU score
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate the mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Set the figure size and create subplots
fig, ax = plt.subplots(figsize=(12, 12))

# Set the width of the bars
bar_width = 0.4

# Plot the F1 scores
f1_bars = ax.barh(np.arange(len(classes_sorted)), f1_sorted, height=bar_width, color='purple', label='F1 Score')

# Plot the mean IoU scores
iou_bars = ax.barh(np.arange(len(classes_sorted)) + bar_width, class_iou_scores_sorted, height=bar_width, color='blue', label='Mean IoU')

# Add F1 scores as text inside each bar
for i, (bar, score) in enumerate(zip(f1_bars, f1_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Add mean IoU scores as text inside each bar
for i, (bar, score) in enumerate(zip(iou_bars, class_iou_scores_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(classes_sorted)) + bar_width / 2)
ax.set_yticklabels(classes_sorted, fontsize=7, fontweight='bold', color='black')

# Set x-axis label and limits
ax.set_xlabel('Scores', fontsize=7, color='red')
ax.set_xlim(0, 1)

# Add class counts as text near the class names
for i, count in enumerate(class_counts_sorted):
    ax.text(-0.1, i + bar_width / 2, f"  ({count})", va='center', ha='right', fontsize=7, fontweight='bold', color='red')

# Set plot title
ax.set_title('F1 Scores and Mean IoU Scores by Class', fontsize=14, fontweight='bold', color='red')

# Remove spines and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(left=False, bottom=False)

# Add a grid to the plot
ax.grid(axis='x', color='lightgray', linestyle='--')

# Show the legend
ax.legend(loc='upper right', fontsize=7)

# Invert the y-axis
plt.gca().invert_yaxis()

# Show the plot
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Create a DataFrame to store the results
results_df_1 = pd.DataFrame({
    'Class': classes_sorted,
    'F1 Score': f1_sorted,
    'Mean IoU': class_iou_scores_sorted,
    'Class Count': class_counts_sorted
})

# Save the results to a CSV file
results_df_1.to_csv('results_1.csv', index=False)

# Display the DataFrame
print(results_df_1)

In [ ]:
##########################################################
################## MSE -2 ################################

In [ ]:
# Perform predictions on the test data
preds = model_resnet_mse_2.predict(test_images)
pred_labels = label_encoder.inverse_transform(np.argmax(preds[0], axis=1))
pred_boxes = preds[1]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate the mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Set the figure size and create subplots
fig, ax = plt.subplots(figsize=(12, 12))

# Set the width of the bars
bar_width = 0.4

# Plot the F1 scores
f1_bars = ax.barh(np.arange(len(classes_sorted)), f1_sorted, height=bar_width, color='purple', label='F1 Score')

# Plot the mean IoU scores
iou_bars = ax.barh(np.arange(len(classes_sorted)) + bar_width, class_iou_scores_sorted, height=bar_width, color='blue', label='Mean IoU')

# Add F1 scores as text inside each bar
for i, (bar, score) in enumerate(zip(f1_bars, f1_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Add mean IoU scores as text inside each bar
for i, (bar, score) in enumerate(zip(iou_bars, class_iou_scores_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(classes_sorted)) + bar_width / 2)
ax.set_yticklabels(classes_sorted, fontsize=7, fontweight='bold', color='black')

# Set x-axis label and limits
ax.set_xlabel('Scores', fontsize=7, color='red')
ax.set_xlim(0, 1)

# Add class counts as text near the class names
for i, count in enumerate(class_counts_sorted):
    ax.text(-0.1, i + bar_width / 2, f"  ({count})", va='center', ha='right', fontsize=7, fontweight='bold', color='red')

# Set plot title
ax.set_title('F1 Scores and Mean IoU Scores by Class', fontsize=14, fontweight='bold', color='red')

# Remove spines and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(left=False, bottom=False)

# Add a grid to the plot
ax.grid(axis='x', color='lightgray', linestyle='--')

# Show the legend
ax.legend(loc='upper right', fontsize=7)

# Invert the y-axis
plt.gca().invert_yaxis()

# Show the plot
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Create a DataFrame to store the results
results_df_2 = pd.DataFrame({
    'Class': classes_sorted,
    'F1 Score': f1_sorted,
    'Mean IoU': class_iou_scores_sorted,
    'Class Count': class_counts_sorted
})

# Save the results to a CSV file
results_df_2.to_csv('results_2.csv', index=False)

# Display the DataFrame
print(results_df_2)

In [ ]:
################################################################
###################### MSE -3 ##################################

In [ ]:
# Perform predictions on the test data
preds = model_top_layers.predict(test_images)
pred_labels = label_encoder.inverse_transform(np.argmax(preds[0], axis=1))
pred_boxes = preds[1]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate the mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Set the figure size and create subplots
fig, ax = plt.subplots(figsize=(12, 12))

# Set the width of the bars
bar_width = 0.4

# Plot the F1 scores
f1_bars = ax.barh(np.arange(len(classes_sorted)), f1_sorted, height=bar_width, color='purple', label='F1 Score')

# Plot the mean IoU scores
iou_bars = ax.barh(np.arange(len(classes_sorted)) + bar_width, class_iou_scores_sorted, height=bar_width, color='blue', label='Mean IoU')

# Add F1 scores as text inside each bar
for i, (bar, score) in enumerate(zip(f1_bars, f1_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Add mean IoU scores as text inside each bar
for i, (bar, score) in enumerate(zip(iou_bars, class_iou_scores_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(classes_sorted)) + bar_width / 2)
ax.set_yticklabels(classes_sorted, fontsize=7, fontweight='bold', color='black')

# Set x-axis label and limits
ax.set_xlabel('Scores', fontsize=7, color='red')
ax.set_xlim(0, 1)

# Add class counts as text near the class names
for i, count in enumerate(class_counts_sorted):
    ax.text(-0.1, i + bar_width / 2, f"  ({count})", va='center', ha='right', fontsize=7, fontweight='bold', color='red')

# Set plot title
ax.set_title('F1 Scores and Mean IoU Scores by Class', fontsize=14, fontweight='bold', color='red')

# Remove spines and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(left=False, bottom=False)

# Add a grid to the plot
ax.grid(axis='x', color='lightgray', linestyle='--')

# Show the legend
ax.legend(loc='upper right', fontsize=7)

# Invert the y-axis
plt.gca().invert_yaxis()

# Show the plot
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Create a DataFrame to store the results
results_df_3 = pd.DataFrame({
    'Class': classes_sorted,
    'F1 Score': f1_sorted,
    'Mean IoU': class_iou_scores_sorted,
    'Class Count': class_counts_sorted
})

# Save the results to a CSV file
results_df_3.to_csv('results_3.csv', index=False)

# Display the DataFrame
print(results_df_3)

In [ ]:
results_df_3

In [ ]:
import shutil

source_file = "/content/results_3.csv"
destination_folder = "/content/drive/MyDrive/project_mark2/"

shutil.copy(source_file, destination_folder)

In [ ]:
source_file = "/content/results_2.csv"
destination_folder = "/content/drive/MyDrive/project_mark2/"

shutil.copy(source_file, destination_folder)

In [ ]:
source_file = "/content/results_1.csv"
destination_folder = "/content/drive/MyDrive/project_mark2/"

shutil.copy(source_file, destination_folder)

In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('results_3.csv')

# Multiply the count column by 5
df['Class Count'] = df['Class Count'] * 5

# Define the image count ranges
ranges = [
    (0, 100),
    (100, 250),
    (250, 450),
    (450, float('inf'))
]

# Create a mapping for the summarized classes
class_mapping = {
    0: 'Class 0-100',
    1: 'Class 100-250',
    2: 'Class 250-450',
    3: 'Class 450+'
}

# Create an empty DataFrame for the new CSV
new_df = pd.DataFrame(columns=['Class', 'Image Count Range', 'Mean IoU', 'Mean F1', 'Total Images'])

# Calculate the mean scores and total images for each summarized class and image count range
for i, (start, end) in enumerate(ranges):
    filtered_df = df[(df['Class Count'] >= start) & (df['Class Count'] < end)]
    mean_iou = filtered_df['Mean IoU'].mean()
    mean_f1 = filtered_df['F1 Score'].mean()
    total_images = filtered_df['Class Count'].sum()
    new_df.loc[i] = [class_mapping[i], f'{start} - {end}', mean_iou, mean_f1, total_images]

# Save the new DataFrame to a CSV file
new_df.to_csv('outputs_3_iou.csv', index=False)


In [ ]:
new_df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file
df = pd.read_csv('outputs_3_iou.csv')

# Set the plot style using Seaborn
sns.set(style='whitegrid')

# Set the color palette
colors = ['#6C8EBF', '#A1C1D9']

# Create a figure and axes
fig, ax = plt.subplots(figsize=(12, 14))

# Plot the mean IoU and mean F1 scores
df.plot(x='Class', y=['Mean IoU', 'Mean F1'], kind='bar', ax=ax, rot=0, legend=True, color=colors)

# Set the title and axis labels
ax.set_title('Mean IoU and Mean F1 Scores by Class (Top then Full Training // Loss = "MSE")', fontsize=16, fontweight='bold')
ax.set_xlabel('Class', fontsize=14)
ax.set_ylabel('Score', fontsize=12)

# Customize the tick labels and font size
ax.tick_params(axis='x', labelrotation=45, labelsize=10)
ax.tick_params(axis='y', labelsize=10)

# Set the legend
ax.legend(['Mean IoU', 'Mean F1'], loc='upper right', fontsize=10)

# Add data labels to the bars
for p in ax.patches:
    x = p.get_x() + p.get_width() / 2.
    y = p.get_height()
    ax.annotate(f'{y:.3f}', (x, y), ha='center', va='center', xytext=(0, 5), textcoords='offset points', fontsize=10, fontweight='bold')

# Remove the top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add a horizontal grid
ax.yaxis.grid(True, linestyle='dashed')

# Adjust the layout
plt.tight_layout()

# Show the plot
plt.show()



In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('results_2.csv')

# Multiply the count column by 5
df['Class Count'] = df['Class Count'] * 5

# Define the image count ranges
ranges = [
    (0, 100),
    (100, 250),
    (250, 450),
    (450, float('inf'))
]

# Create a mapping for the summarized classes
class_mapping = {
    0: 'Class 0-100',
    1: 'Class 100-250',
    2: 'Class 250-450',
    3: 'Class 450+'
}

# Create an empty DataFrame for the new CSV
new_df = pd.DataFrame(columns=['Class', 'Image Count Range', 'Mean IoU', 'Mean F1', 'Total Images'])

# Calculate the mean scores and total images for each summarized class and image count range
for i, (start, end) in enumerate(ranges):
    filtered_df = df[(df['Class Count'] >= start) & (df['Class Count'] < end)]
    mean_iou = filtered_df['Mean IoU'].mean()
    mean_f1 = filtered_df['F1 Score'].mean()
    total_images = filtered_df['Class Count'].sum()
    new_df.loc[i] = [class_mapping[i], f'{start} - {end}', mean_iou, mean_f1, total_images]

# Save the new DataFrame to a CSV file
new_df.to_csv('outputs_2_iou.csv', index=False)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file
df = pd.read_csv('outputs_2_iou.csv')

# Set the plot style using Seaborn
sns.set(style='whitegrid')

# Set the color palette
colors = ['#6C8EBF', '#A1C1D9']

# Create a figure and axes
fig, ax = plt.subplots(figsize=(12, 14))

# Plot the mean IoU and mean F1 scores
df.plot(x='Class', y=['Mean IoU', 'Mean F1'], kind='bar', ax=ax, rot=0, legend=True, color=colors)

# Set the title and axis labels
ax.set_title('Mean IoU and Mean F1 Scores by Class (Top-only Training // Loss = "MSE")', fontsize=16, fontweight='bold')
ax.set_xlabel('Class', fontsize=14)
ax.set_ylabel('Score', fontsize=12)

# Customize the tick labels and font size
ax.tick_params(axis='x', labelrotation=45, labelsize=10)
ax.tick_params(axis='y', labelsize=10)

# Set the legend
ax.legend(['Mean IoU', 'Mean F1'], loc='upper right', fontsize=10)

# Add data labels to the bars
for p in ax.patches:
    x = p.get_x() + p.get_width() / 2.
    y = p.get_height()
    ax.annotate(f'{y:.3f}', (x, y), ha='center', va='center', xytext=(0, 5), textcoords='offset points', fontsize=10, fontweight='bold')

# Remove the top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add a horizontal grid
ax.yaxis.grid(True, linestyle='dashed')

# Adjust the layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('results_1.csv')

# Multiply the count column by 5
df['Class Count'] = df['Class Count'] * 5

# Define the image count ranges
ranges = [
    (0, 100),
    (100, 250),
    (250, 450),
    (450, float('inf'))
]

# Create a mapping for the summarized classes
class_mapping = {
    0: 'Class 0-100',
    1: 'Class 100-250',
    2: 'Class 250-450',
    3: 'Class 450+'
}

# Create an empty DataFrame for the new CSV
new_df = pd.DataFrame(columns=['Class', 'Image Count Range', 'Mean IoU', 'Mean F1', 'Total Images'])

# Calculate the mean scores and total images for each summarized class and image count range
for i, (start, end) in enumerate(ranges):
    filtered_df = df[(df['Class Count'] >= start) & (df['Class Count'] < end)]
    mean_iou = filtered_df['Mean IoU'].mean()
    mean_f1 = filtered_df['F1 Score'].mean()
    total_images = filtered_df['Class Count'].sum()
    new_df.loc[i] = [class_mapping[i], f'{start} - {end}', mean_iou, mean_f1, total_images]

# Save the new DataFrame to a CSV file
new_df.to_csv('outputs_1_iou.csv', index=False)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file
df = pd.read_csv('outputs_1_iou.csv')

# Set the plot style using Seaborn
sns.set(style='whitegrid')

# Set the color palette
colors = ['#6C8EBF', '#A1C1D9']

# Create a figure and axes
fig, ax = plt.subplots(figsize=(12, 14))

# Plot the mean IoU and mean F1 scores
df.plot(x='Class', y=['Mean IoU', 'Mean F1'], kind='bar', ax=ax, rot=0, legend=True, color=colors)

# Set the title and axis labels
ax.set_title('Mean IoU and Mean F1 Scores by Class (Full Training // Loss = "MSE")', fontsize=16, fontweight='bold')
ax.set_xlabel('Class', fontsize=14)
ax.set_ylabel('Score', fontsize=12)

# Customize the tick labels and font size
ax.tick_params(axis='x', labelrotation=45, labelsize=10)
ax.tick_params(axis='y', labelsize=10)

# Set the legend
ax.legend(['Mean IoU', 'Mean F1'], loc='upper right', fontsize=10)

# Add data labels to the bars
for p in ax.patches:
    x = p.get_x() + p.get_width() / 2.
    y = p.get_height()
    ax.annotate(f'{y:.3f}', (x, y), ha='center', va='center', xytext=(0, 5), textcoords='offset points', fontsize=10, fontweight='bold')

# Remove the top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add a horizontal grid
ax.yaxis.grid(True, linestyle='dashed')

# Adjust the layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import shutil

source_file = "/content/outputs_3_iou.csv"
destination_folder = "/content/drive/MyDrive/project_mark2/"

shutil.copy(source_file, destination_folder)

In [ ]:
import shutil

source_file = "/content/outputs_2_iou.csv"
destination_folder = "/content/drive/MyDrive/project_mark2/"

shutil.copy(source_file, destination_folder)

In [ ]:
import shutil

source_file = "/content/outputs_1_iou.csv"
destination_folder = "/content/drive/MyDrive/project_mark2/"

shutil.copy(source_file, destination_folder)

In [ ]:
# # Save test_carside_df to CSV
# test_carside_df.to_csv(os.path.join(parent_dir, "test_carside_df.csv"), index=False)

# # Save val_carside_df to CSV
# val_carside_df.to_csv(os.path.join(parent_dir, "val_carside_df.csv"), index=False)

# # Save carside_df to CSV
# carside_df.to_csv(os.path.join(parent_dir, "carside_df.csv"), index=False)


In [ ]:
file_path = '/content/drive/My Drive/project_mark2/test_carside_df.csv'
test_carside_df = pd.read_csv(file_path)

In [ ]:
file_path = '/content/drive/My Drive/project_mark2/val_carside_df.csv'
val_carside_df = pd.read_csv(file_path)

In [ ]:
file_path = '/content/drive/My Drive/project_mark2/carside_df.csv'
carside_df = pd.read_csv(file_path)

In [ ]:
column_names = ['Image Path', 'Class Name', 'Image Name', 'Bounding Box', 'Height', 'Width', 'label',
                'normalized', 'Xmin', 'Ymin', 'Xmax', 'Ymax']


test_carside_df.columns = column_names
val_carside_df.columns = column_names
carside_df.columns = column_names

In [ ]:
def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    image = image.astype(np.float32) / 255.0
    return image

In [ ]:
def plot_images_with_bboxes(df):
    fig, axes = plt.subplots(1, 5, figsize=(20, 4))
    for i, row in df.head(5).iterrows():
        image_path = row['Image Path']
        class_name = row['Class Name']
        image_name = row['Image Name']
        xmin, ymin, xmax, ymax = row['Xmin']*224, row['Ymin']*224, row['Xmax']*224, row['Ymax']*224

        image = load_image(image_path)

        ax = axes[i]
        ax.imshow(image)
        ax.axis('off')

        # Add bounding box
        bbox = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(bbox)

        ax.set_title(f'Class: {class_name}\nImage: {image_name}')

    plt.show()

# Plot images with bounding boxes for val_df, test_df, train_df
plot_images_with_bboxes(val_carside_df)
plot_images_with_bboxes(test_carside_df)
plot_images_with_bboxes(carside_df)

In [ ]:
####################################################
############## MOST IMPORTANT PART #################
####################################################
def resize_image(image, target_size):
    return cv2.resize(image, target_size)

def plot_images(images, boxes, labels, dataset_name):
    fig, axes = plt.subplots(1, len(images), figsize=(12, 4))
    for i, ax in enumerate(axes):
        ax.imshow(images[i])
        xmin, ymin, xmax, ymax = boxes[i]
        ax.add_patch(plt.Rectangle((xmin*224, ymin*224), xmax*224 - xmin*224, ymax*224 - ymin*224, fill=False, color='red', linewidth=2))
        ax.set_title(labels[i])
        ax.axis('off')
    plt.suptitle(f'{dataset_name} Images with Bounding Boxes')
    plt.show()

# Append data from val_df into arrays
val_images = []
val_boxes = []
val_labels = []

for index, row in val_carside_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    val_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    val_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    val_labels.append(label)

# Convert the lists to NumPy arrays
val_images = np.array(val_images, dtype='float32')
val_boxes = np.array(val_boxes, dtype='float32')
val_labels = np.array(val_labels)

# Normalize the images
val_images = val_images / 255.0



# Append data from train_df into arrays
train_images = []
train_boxes = []
train_labels = []

for index, row in carside_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    train_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    train_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    train_labels.append(label)

# Convert the lists to NumPy arrays
train_images = np.array(train_images, dtype='float32')
train_boxes = np.array(train_boxes, dtype='float32')
train_labels = np.array(train_labels)

# Normalize the images
train_images = train_images / 255.0



# Append data from test_df into arrays
test_images = []
test_boxes = []
test_labels = []

for index, row in test_carside_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    test_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    test_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    test_labels.append(label)

# Convert the lists to NumPy arrays
test_images = np.array(test_images, dtype='float32')
test_boxes = np.array(test_boxes, dtype='float32')
test_labels = np.array(test_labels)

# Normalize the images
test_images = test_images / 255.0

In [ ]:
print("len -- val images: ", len(val_images))
print("len -- test images: ", len(test_images))
print("len -- train images: ", len(train_images))


print("type -- test images: ", type(test_images))
print("type -- train images: ", type(train_images))
print("type -- val images: ", type(val_images))


In [ ]:
print("shape test_boxes: ", test_boxes.shape)
print("shape val_boxes: ", val_boxes.shape)
print("shape train_boxes: ", train_boxes.shape)


In [ ]:
test_labels

In [ ]:
train_labels

In [ ]:
test_boxes

## **Carside -- Imagenet Trainable = False Test - IoU**

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Convert the lists to NumPy arrays
test_images = np.array(test_images, dtype='float32')
test_boxes = np.array(test_boxes, dtype='float32')
test_labels = np.array(test_labels)

train_images = np.array(train_images, dtype='float32')
train_boxes = np.array(train_boxes, dtype='float32')
train_labels = np.array(train_labels)

val_images = np.array(val_images, dtype='float32')
val_boxes = np.array(val_boxes, dtype='float32')
val_labels = np.array(val_labels)

In [ ]:
# Perform label encoding on the target labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Convert the encoded labels to one-hot encoding
num_classes = len(label_encoder.classes_)
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=num_classes)
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=num_classes)
test_labels_onehot = to_categorical(test_labels_encoded, num_classes=num_classes)


In [ ]:
# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)


flatten = Flatten()(base_model.output)

bboxHead = Dense(1024, activation="relu")(flatten)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(256, activation="relu")(bboxHead)
bboxHead = Dense(128, activation="relu")(bboxHead)
bbox_output = Dense(4, activation='linear', name='bbox_output')(bboxHead)



carside_test_1 = Model(inputs=input_layer, outputs=[bbox_output])


In [ ]:
# Loss function
losses = {
    'bbox_output': 'mse'
}

In [ ]:
learning_rate = 1e-4

carside_test_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=losses)

carside_test_1.summary()

In [ ]:
from keras.callbacks import EarlyStopping

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

carside_test_1_history  = carside_test_1.fit(train_images, {'bbox_output': train_boxes},
                                                   validation_data=(val_images, {'bbox_output': val_boxes}),
                                                   epochs=300, batch_size=32, callbacks=[callback])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get the loss values from the history object
loss = carside_test_1_history.history['loss']
val_loss = carside_test_1_history.history['val_loss']

# Plot the loss and validation loss
epochs = range(1, len(loss) + 1)
plt.semilogy(epochs, loss, 'b', label='Training Loss')
plt.semilogy(epochs, val_loss, 'r', label='Validation Loss')

# Customize the plot appearance
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.legend()

# Add labels to the first and last data points
plt.text(1, loss[0], f'{loss[0]:.4f}', ha='center', va='bottom', fontsize=8, color='b')
plt.text(1, val_loss[0], f'{val_loss[0]:.4f}', ha='center', va='top', fontsize=8, color='r')
plt.text(len(loss), loss[-1], f'{loss[-1]:.4f}', ha='center', va='bottom', fontsize=8, color='b')
plt.text(len(val_loss), val_loss[-1], f'{val_loss[-1]:.4f}', ha='center', va='top', fontsize=8, color='r')

# Plot markers for the first and last data points
plt.plot(1, loss[0], marker='o', markersize=4, color='b')
plt.plot(1, val_loss[0], marker='o', markersize=4, color='r')
plt.plot(len(loss), loss[-1], marker='o', markersize=4, color='b')
plt.plot(len(val_loss), val_loss[-1], marker='o', markersize=4, color='r')

plt.tight_layout()
plt.show()


In [ ]:
# Save the model
carside_test_1.save('/content/drive/MyDrive/project_mark2/carside_test_1_model.h5')

In [ ]:
import pickle

# Save the history
with open('/content/drive/MyDrive/project_mark2/carside_test_1_history.pkl', 'wb') as file:
    pickle.dump(carside_test_1_history.history, file)

In [ ]:
test_carside_df.head()

In [ ]:
import matplotlib.pyplot as plt

# Extract loss values from the history
loss = carside_test_1_history.history['loss']
val_loss = carside_test_1_history.history['val_loss']

# Print first and last values
print("Training Loss:")
print(f"  First value: {loss[0]:.4f}")
print(f"   Last value: {loss[-1]:.4f}")
print("Validation Loss:")
print(f"  First value: {val_loss[0]:.4f}")
print(f"   Last value: {val_loss[-1]:.4f}")

# Calculate min and max values
loss_min = min(loss)
loss_max = max(loss)
val_loss_min = min(val_loss)
val_loss_max = max(val_loss)

# Plot the losses
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training Loss and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Annotate min and max values
plt.annotate(f"Min: {loss_min:.4f}", xy=(epochs[loss.index(loss_min)], loss_min),
             xytext=(10, 20), textcoords='offset points',
             arrowprops=dict(arrowstyle="->", lw=1.5))
plt.annotate(f"Max: {loss_max:.4f}", xy=(epochs[loss.index(loss_max)], loss_max),
             xytext=(-70, -30), textcoords='offset points',
             arrowprops=dict(arrowstyle="->", lw=1.5))
plt.annotate(f"Min: {val_loss_min:.4f}", xy=(epochs[val_loss.index(val_loss_min)], val_loss_min),
             xytext=(-70, 20), textcoords='offset points',
             arrowprops=dict(arrowstyle="->", lw=1.5))
plt.annotate(f"Max: {val_loss_max:.4f}", xy=(epochs[val_loss.index(val_loss_max)], val_loss_max),
             xytext=(10, -30), textcoords='offset points',
             arrowprops=dict(arrowstyle="->", lw=1.5))

# Adjust plot layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Save the history object to a file using pickle
with open('/content/drive/MyDrive/project_mark2/carside_test_1_history.pkl', 'wb') as file:
    pickle.dump(carside_test_1_history.history, file)

In [ ]:
import shutil

file_path = '/content/drive/My Drive/project_mark2/carside_test_1_model.h5'
destination_path = 'carside_test_1_model.h5'

shutil.copyfile(file_path, destination_path)

In [ ]:
from tensorflow import keras

model = keras.models.load_model('carside_test_1_model.h5')

In [ ]:
###################### ^^^^^^^^^^^ after training ^^^^^^^^^^^^^^^^^^^ #########################

In [ ]:
test_boxes[:5]

In [ ]:
test_carside_df

In [ ]:
import pandas as pd

# Assuming you have the test_images array and your_model variable

# Create an empty list to store bounding box information
bbox_data = []

# Iterate over the test_images array
for image in test_images:
    # Perform inference using your model to get the predicted bounding box coordinates


    # Reshape the image to match the expected input shape of the model
    reshaped_image = np.expand_dims(image, axis=0)

    # Perform inference using your model to get the predicted bounding box coordinates
    predicted_bbox = model.predict(reshaped_image)
    print(predicted_bbox)

    # Append the image path and predicted bounding box coordinates to the bbox_data list
    bbox_data.append({
        'Image Path': image_path,  # Replace with the actual image path if available
        'p_Xmin': predicted_bbox[0][0],
        'p_Ymin': predicted_bbox[0][1],
        'p_Xmax': predicted_bbox[0][2],
        'p_Ymax': predicted_bbox[0][3]
    })

# Create a dataframe from the bbox_data list
bounding_boxes_df = pd.DataFrame(bbox_data)

# Print the dataframe
print(bounding_boxes_df)

In [ ]:
bounding_boxes_df

In [ ]:
import pandas as pd

# Assuming you have the predicted bounding boxes dataframe stored in pred_df
# and the original dataframe stored in test_carside_df

merged_df = test_carside_df.merge(bounding_boxes_df, on="Image Path", how="inner")

In [ ]:
merged_df

In [ ]:
import pandas as pd

# Function to calculate IoU
def calculate_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.
    :param bb1: Bounding box 1 [xmin, ymin, xmax, ymax]
    :param bb2: Bounding box 2 [xmin, ymin, xmax, ymax]
    :return: IoU value
    """
    # Calculate the intersection area
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Calculate the union area
    bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
    bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])
    union_area = bb1_area + bb2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area
    return iou

# Calculate mean IoU for each image
def calculate_mean_iou(df):
    """
    Calculate the mean Intersection over Union (IoU) for each image in the dataframe.
    :param df: Dataframe containing bounding box information
    :return: Series with mean IoU for each image
    """
    mean_ious = []
    unique_images = df['Image Name'].unique()

    for image_name in unique_images:
        image_df = df[df['Image Name'] == image_name]

        ious = []
        for i in range(len(image_df)):
            row = image_df.iloc[i]
            bb1 = [row['p_Xmin'], row['p_Ymin'], row['p_Xmax'], row['p_Ymax']]
            bb2 = [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']]
            iou = calculate_iou(bb1, bb2)
            ious.append(iou)

        mean_iou = sum(ious) / len(ious)
        mean_ious.append(mean_iou)

    return pd.Series(mean_ious, index=unique_images)


# Calculate mean IoU for each image
mean_ious = calculate_mean_iou(merged_df)
print(mean_ious)

In [ ]:
import pandas as pd

# Calculate the mean IoU for each image
merged_df['mean_IoU'] = merged_df.apply(lambda row: (row['p_Xmax'] - row['p_Xmin']) * (row['p_Ymax'] - row['p_Ymin']), axis=1)

# Group the dataframe by image name and calculate the mean IoU
mean_IoU_per_image = merged_df.groupby('Image Name')['mean_IoU'].mean().reset_index()

# Merge the mean IoU values back to the original dataframe
merged_df = pd.merge(merged_df, mean_IoU_per_image, on='Image Name')

# Display the updated dataframe
print(merged_df)

In [ ]:
merged_df

In [ ]:
import pandas as pd

# Calculate the IoU for each bounding box in the x-axis and y-axis
merged_df['IoU_x'] = merged_df.apply(lambda row: row['p_Xmax'] - row['p_Xmin'], axis=1)
merged_df['IoU_y'] = merged_df.apply(lambda row: row['p_Ymax'] - row['p_Ymin'], axis=1)

# Calculate the total mean IoU
mean_IoU = (merged_df['IoU_x'].mean() + merged_df['IoU_y'].mean()) / 2

# Display the total mean IoU
print("Total Mean IoU:", mean_IoU)

In [ ]:
merged_df

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Function to plot normalized bounding boxes on images
def plot_normalized_bounding_boxes(df):
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Read image path
        image_path = row['Image Path']

        # Load image as RGB
        image = Image.open(image_path).convert("RGB")

        # Create figure and axes
        fig, ax = plt.subplots(1)

        # Display the image
        ax.imshow(image)

        # Retrieve normalized bounding box coordinates
        xmin = row['p_Xmin'] * row['Width']
        ymin = row['p_Ymin'] * row['Height']
        xmax = row['p_Xmax'] * row['Width']
        ymax = row['p_Ymax'] * row['Height']

        # Create a Rectangle patch for the predicted bounding box
        pred_box = patches.Rectangle((xmin, ymin), (xmax - xmin), (ymax - ymin), linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(pred_box)

        # Create a Rectangle patch for the original bounding box
        orig_box = patches.Rectangle((row['Xmin'], row['Ymin']), (row['Xmax'] - row['Xmin']), (row['Ymax'] - row['Ymin']), linewidth=2, edgecolor='g', facecolor='none')
        ax.add_patch(orig_box)

        # Set title
        ax.set_title("Image: {}".format(row['Image Name']))

        # Show the plot
        plt.show()

# Call the function with your DataFrame
plot_normalized_bounding_boxes(merged_df)


In [ ]:
test_images

In [ ]:
# Append data from test_df into arrays
test_images = []
test_boxes = []
test_labels = []

for index, row in test_carside_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    test_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    test_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    test_labels.append(label)

# Convert the lists to NumPy arrays
test_images = np.array(test_images, dtype='float32')
test_boxes = np.array(test_boxes, dtype='float32')
test_labels = np.array(test_labels)

# Normalize the images
test_images = test_images / 255.0



In [ ]:
test_images

In [ ]:
test_boxes

In [ ]:
test_images[1]

In [ ]:
import matplotlib.pyplot as plt
def plot_images(images):
    for i in range(len(images)):
        plt.imshow(images[i])
        plt.axis('off')
        plt.show()


plot_images(test_images)

In [ ]:
test_boxes


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_images_with_boxes(images, boxes):
    for i in range(len(images)):
        image = images[i]
        box = boxes[i]

        # Create figure and axes
        fig, ax = plt.subplots()

        # Display the image
        ax.imshow(image)

        # Create a rectangle patch for the bounding box
        rect = patches.Rectangle((box[1]*255, box[0]*255), box[3]*255-box[1]*255, box[2]*255-box[0]*255, linewidth=2, edgecolor='r', facecolor='none')

        # Add the rectangle to the plot
        ax.add_patch(rect)

        # Remove axis ticks and labels
        ax.axis('off')

        # Show the plot
        plt.show()



plot_images_with_boxes(test_images, test_boxes)

In [ ]:
import matplotlib.pyplot as plt

def plot_images_with_boxes(images, boxes):
    for i in range(len(images)):
        image = images[i]
        box = boxes[i]

        # Multiply bounding box values by 255
        xmin = box[0] * 255
        ymin = box[1] * 255
        xmax = box[2] * 255
        ymax = box[3] * 255

        # Plot the image
        plt.imshow(image)

        # Plot the bounding box
        rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                             edgecolor='r', facecolor='none')
        plt.gca().add_patch(rect)

        plt.show()


plot_images_with_boxes(test_images, test_boxes)

In [ ]:
merged_df



In [ ]:
import cv2

def plot_images_with_boxes(data):
    for i, row in data.iterrows():
        image_path = row['Image Path']
        class_name = row['Class Name']
        image_name = row['Image Name']
        xmin = int(row['Xmin'] * 255)
        ymin = int(row['Ymin'] * 255)
        xmax = int(row['Xmax'] * 255)
        ymax = int(row['Ymax'] * 255)
        p_xmin = int(row['p_Xmin'] * 255)
        p_ymin = int(row['p_Ymin'] * 255)
        p_xmax = int(row['p_Xmax'] * 255)
        p_ymax = int(row['p_Ymax'] * 255)

        # Load the image
        image = cv2.imread(image_path)

        # Draw the original bounding box
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)

        # Draw the predicted bounding box
        cv2.rectangle(image, (p_xmin, p_ymin), (p_xmax, p_ymax), (0, 255, 0), 2)

        # Display the image
        plt.imshow(data)

In [ ]:
plot_images_with_boxes(merged_df)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Define a function to plot images with bounding boxes
def plot_images_with_bounding_boxes(dataframe):
    fig, axs = plt.subplots(nrows=len(dataframe), figsize=(8, 8 * len(dataframe)))

    for idx, row in dataframe.iterrows():
        # Load the image
        image_path = row['Image Path']
        image = Image.open(image_path)

        # Plot the image
        ax = axs[idx]
        ax.imshow(image)
        ax.axis('off')

        # Get ground truth bounding box coordinates
        xmin_gt = row['Xmin'] * row['Width']
        ymin_gt = row['Ymin'] * row['Height']
        xmax_gt = row['Xmax'] * row['Width']
        ymax_gt = row['Ymax'] * row['Height']

        # Calculate ground truth bounding box width and height
        width_gt = xmax_gt - xmin_gt
        height_gt = ymax_gt - ymin_gt

        # Create a ground truth rectangle patch
        rect_gt = patches.Rectangle((xmin_gt, ymin_gt), width_gt, height_gt, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect_gt)

        # Get predicted bounding box coordinates
        xmin_pred = row['p_Xmin'] * row['Width']
        ymin_pred = row['p_Ymin'] * row['Height']
        xmax_pred = row['p_Xmax'] * row['Width']
        ymax_pred = row['p_Ymax'] * row['Height']

        # Calculate predicted bounding box width and height
        width_pred = xmax_pred - xmin_pred
        height_pred = ymax_pred - ymin_pred

        # Create a predicted rectangle patch
        rect_pred = patches.Rectangle((xmin_pred, ymin_pred), width_pred, height_pred, linewidth=2, edgecolor='g', facecolor='none')
        ax.add_patch(rect_pred)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Call the function with your merged_df
plot_images_with_bounding_boxes(merged_df)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Define a function to plot images with bounding boxes
def plot_images_with_bounding_boxes(dataframe):
    for _, row in dataframe.iterrows():
        # Load the image
        image_path = row['Image Path']
        image = Image.open(image_path)

        # Create a figure and axis
        fig, ax = plt.subplots()

        # Plot the image
        ax.imshow(image)
        ax.axis('off')

        # Get bounding box coordinates
        xmin = row['Xmin']
        ymin = row['Ymin']
        xmax = row['Xmax']
        ymax = row['Ymax']

        # Calculate bounding box width and height
        width = xmax - xmin
        height = ymax - ymin

        # Create a rectangle patch
        rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='r', facecolor='none')

        # Add the rectangle patch to the plot
        ax.add_patch(rect)

        # Show the plot
        plt.show()

# Call the function with your merged_df
plot_images_with_bounding_boxes(merged_df)